In [1]:
import random

import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.read_csv("../data/csv/achmusik.csv")

data = data[data.columns[:-2]]
# Getting the decimals right -- commas to points and no more Nones
data = data.set_index(["genre", "sub_genre", "artist", "album", "song"])
data.fillna(value="", inplace=True)

for i in range(data.columns.size):
    data[data.columns[i]] = pd.to_numeric(data[data.columns[i]], errors='coerce')

data

Qu  \
genre   sub_genre        artist       album                   song                          
Electro Drum'n'Bass      1991         Pleasure                Pleasure               6.40   
                                      Power                   Power                  5.50   
Reggae  NaN              10cc         Bloody Tourists         Dreadlock Holiday      7.25   
Rock    Soft Rock        10cc         The Original Soundtrack I'm Not In Love        4.50   
Hip-Hop NaN              13 Block     BLO                     93 Gangstérisme        1.00   
...                                                                                   ...   
Metal   Negro Spirituals Zeal & Ardor Zeal & Ardor            Hold Your Head Low     7.70   
        Black            Zeal & Ardor Zeal & Ardor            J-M-B                  7.00   
Electro Ambient          Zeal & Ardor Zeal & Ardor            A-H-I-L                5.90   
        Parody           TwitchHits   Assclap Right Version   Assclap Right Version  7.80   
Pop     NaN              Peter Allen  Taught By Experts       I Go To Rio            7.40   

                                                                                      Gr  \
genre   sub_genre        artist       album                   song                         
Electro Drum'n'Bass      1991         Pleasure                Pleasure               NaN   
                                      Power                   Power                  NaN   
Reggae  NaN              10cc         Bloody Tourists         Dreadlock Holiday      7.0   
Rock    Soft Rock        10cc         The Original Soundtrack I'm Not In Love        7.6   
Hip-Hop NaN              13 Block     BLO                     93 Gangstérisme        NaN   
...                                                                                  ...   
Metal   Negro Spirituals Zeal & Ardor Zeal & Ardor            Hold Your Head Low     NaN   
        Black            Zeal & Ardor Zeal & Ardor            J-M-B                  NaN   
Electro Ambient          Zeal & Ardor Zeal & Ardor            A-H-I-L                NaN   
        Parody           TwitchHits   Assclap Right Version   Assclap Right Version  NaN   
Pop     NaN              Peter Allen  Taught By Experts       I Go To Rio            NaN   

                                                                                       Vi  \
genre   sub_genre        artist       album                   song                          
Electro Drum'n'Bass      1991         Pleasure                Pleasure               5.75   
                                      Power                   Power                  6.00   
Reggae  NaN              10cc         Bloody Tourists         Dreadlock Holiday      7.00   
Rock    Soft Rock        10cc         The Original Soundtrack I'm Not In Love        6.00   
Hip-Hop NaN              13 Block     BLO                     93 Gangstérisme        6.00   
...                                                                                   ...   
Metal   Negro Spirituals Zeal & Ardor Zeal & Ardor            Hold Your Head Low      NaN   
        Black            Zeal & Ardor Zeal & Ardor            J-M-B                   NaN   
Electro Ambient          Zeal & Ardor Zeal & Ardor            A-H-I-L                 NaN   
        Parody           TwitchHits   Assclap Right Version   Assclap Right Version   NaN   
Pop     NaN              Peter Allen  Taught By Experts       I Go To Rio             NaN   

                                                                                     Ro  \
genre   sub_genre        artist       album                   song                        
Electro Drum'n'Bass      1991         Pleasure                Pleasure              NaN   
                                      Power                   Power                 NaN   
Reggae  NaN              10cc         Bloody Tourists         Dreadlock Holiday     NaN   
Rock    Soft Ro

In [18]:
# TOP AND BOTTOM ARTISTS or ALBUMS
guy = "Vi"
thing = "song"
amnt = 5

qu_df = pd.DataFrame(columns=["mean", "ok"])
qu_df["ok"] = data[guy].dropna().groupby(thing).size() > 4
qu_df["mean"] = data[guy].dropna().groupby(thing).mean()

qu_df = qu_df[qu_df["ok"] == True].sort_values(by="mean", ascending=False)
print(qu_df.head(amnt), qu_df.tail(amnt))

for i in qu_df.head(amnt).index:
    print(i)
print("AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA")
for i in qu_df.tail(amnt).index:
    print(i)

        mean    ok
song              
Intro  6.175  True         mean    ok
song              
Intro  6.175  True
Intro
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
Intro


In [12]:
# To get amount of grades out of total for genres and person
guys = data.columns
genres_df = pd.DataFrame(index=data.index.get_level_values("genre").unique(), columns=[guy])

for guy in guys:
    for i in data.index.get_level_values("genre").unique():
        if i == "nan":
            pass
        if len(data[data.index.get_level_values("genre")  == i]) > 0:
            ratio = len(data.iloc[data.index.get_level_values("genre") == i][guy].dropna()) / len(data[data.index.get_level_values("genre")  == i])
            genres_df.at[i, guy] = int(ratio*100)

cols = ["Qu", "Gr", "Vi", "Ro", "Sa", "Gl", "Cl", "Lu", "Gë", "Et", "Ti"]
genres_df = genres_df.drop(genres_df.iloc[-1]).astype(int)
genres_df[cols].reindex(data.index.get_level_values("genre").value_counts().index)

KeyError: '[100 33.0 100.0 66.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0] not found in axis'

In [13]:
genres_df[cols].reindex(data.index.get_level_values("genre").value_counts().index).astype("int")

,Qu,Gr,Vi,Ro,Sa,Gl,Cl,Lu,Gë,Et,Ti
Rock,86,43,72,16,10,1,2,0,10,0,7
Metal,90,67,25,28,18,1,4,1,4,0,7
Electro,86,14,77,17,13,0,2,1,4,0,11
Hip-Hop,75,7,80,7,1,0,1,7,2,12,8
Pop,83,26,76,24,7,4,4,0,12,0,10
Jazz,99,4,75,10,9,1,0,0,2,0,6
Chanson,93,38,64,15,0,2,8,0,9,4,15
Funk,100,24,53,19,2,3,0,0,11,2,8
Reggae,93,0,48,62,0,0,51,0,0,0,4
Disco,100,38,70,23,16,0,2,0,23,3,8


In [13]:
len(data.iloc[data.index.get_level_values("genre") == "Reggae"]["Qu"].dropna()) / len(data[data.index.get_level_values("genre")  == "Reggae"])

1.0